In [1]:
%load_ext autoreload
%autoreload 2
import sys
from pathlib import Path
path = str(Path.cwd().parent)
print(path)
sys.path.insert(1, path)

import numpy as np
import pandas as pd

/home/joaquin/Documents/GitHub/skforecast


In [ ]:
from sklearn.preprocessing import OrdinalEncoder
from skforecast.experimental import FastOrdinalEncoder
from skforecast.recursive import ForecasterRecursiveMultiSeries
from lightgbm import LGBMRegressor
import numpy as np
import pandas as pd
import skforecast
skforecast.__version__  


In [3]:
products = [f"series_{i}" for i in range(1, 5001)]
data = {
    'product': np.repeat(products, 1000),
    'price': np.random.uniform(10, 100, 5000000),
}
df = pd.DataFrame(data)
display(df)

df_new_category = df.copy()
df_new_category['product'] = df_new_category['product'].replace('series_1', 'new_series')

,product,price
0,series_1,82.885649
1,series_1,85.119064
2,series_1,57.218043
3,series_1,49.255848
4,series_1,13.557479
...,...,...
4999995,series_5000,83.531397
4999996,series_5000,24.537852
4999997,series_5000,23.007796
4999998,series_5000,43.026974


In [4]:
# Fit and transform

In [5]:
%%timeit -r 10 -n 10
encoder = OrdinalEncoder(
    categories = 'auto',
    dtype      = int
).set_output(transform='pandas')
encoder.fit(df[['product']])
encoded_values = encoder.transform(df[['product']])

1.26 s ± 236 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [6]:
%%timeit -r 10 -n 10
encoder = FastOrdinalEncoder()
encoder.fit(categories=products)
encoded_values = encoder.transform(df[['product']])

418 ms ± 63.8 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [7]:
# Inverse transform

In [8]:
encoder = OrdinalEncoder(
    categories = 'auto',
    dtype      = int
).set_output(transform='pandas')
encoder.fit(df[['product']])
encoded_values = encoder.transform(df[['product']])

In [9]:
%%timeit -r 10 -n 10
encoder.inverse_transform(encoded_values[['product']])

97.8 ms ± 7.78 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [10]:
encoder = FastOrdinalEncoder()
encoder.fit(categories=products)
encoded_values = encoder.transform(df[['product']])

In [11]:
%%timeit -r 10 -n 10
encoder.inverse_transform(encoded_values[['product']])

450 ms ± 39.7 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [12]:
encoder = OrdinalEncoder(
    categories = 'auto',
    dtype      = int
).set_output(transform='pandas')
encoder.fit(df[['product']])
encoded_values = encoder.transform(df[['product']])
display(encoded_values)

inverse_values = encoder.inverse_transform(encoded_values[['product']])
display(inverse_values.ravel())

,product
0,0
1,0
2,0
3,0
4,0
...,...
4999995,4447
4999996,4447
4999997,4447
4999998,4447


array(['series_1', 'series_1', 'series_1', ..., 'series_5000',
       'series_5000', 'series_5000'], shape=(5000000,), dtype=object)

In [13]:
encoder = FastOrdinalEncoder()
encoder.fit(categories=products)
encoded_values_fast = encoder.transform(df['product'])
display(encoded_values_fast)
inverse_values_fast = encoder.inverse_transform(encoded_values_fast[['product']])
display(inverse_values_fast.ravel())

,product
0,0
1,0
2,0
3,0
4,0
...,...
4999995,4447
4999996,4447
4999997,4447
4999998,4447


array([np.str_('series_1'), np.str_('series_1'), np.str_('series_1'), ...,
       np.str_('series_5000'), np.str_('series_5000'),
       np.str_('series_5000')], shape=(5000000,), dtype=object)

In [6]:
from skforecast.recursive import ForecasterRecursiveMultiSeries
from lightgbm import LGBMRegressor
import numpy as np
import pandas as pd

forecaster = ForecasterRecursiveMultiSeries(
    regressor=LGBMRegressor(n_estimators=10, random_state=123),
    lags=24,
    encoding='ordinal',
)

products = [f"series_{i}" for i in range(1, 5001)]
datetime = pd.date_range(start='2020-01-01', periods=1000, freq='h')
series = {
    'product': np.repeat(products, 1000),
    'value': np.random.uniform(10, 100, 5000000),
    'datetime': np.tile(datetime, 5000)
}
series = pd.DataFrame(series)
series_dict = {}
for k, v in series.groupby('product'):
    series_dict[k] = v.set_index('datetime')['value'].asfreq('h', fill_value=np.nan).rename(k)
    series_dict[k].index.name = None

In [5]:
%%timeit -r 10 -n 10

X, y = forecaster.create_train_X_y(series=series_dict)

3.05 s ± 97.3 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)


In [ ]:
%%timeit -r 10 -n 10
                                                                                                                                                                                        
X, y = forecaster.create_train_X_y(series=series_dict)

6.46 s ± 178 ms per loop (mean ± std. dev. of 10 runs, 10 loops each)
